# 1. Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

In [ ]:
# Horizontal Scrolling
import io
import base64
from IPython.display import HTML, display

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.ndimage import binary_dilation
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as MAE
from pandas import DataFrame, Series, Timedelta

In [ ]:
from utils.loading import load_all_raw_data
from utils.preprocessing import process_all_dfs
from utils.merging import merge_all_dfs
from utils.feature_engineering import *

In [ ]:
pd.set_option(
    "display.float_format",
    lambda x: f"{x:.2e}" if abs(x) < 0.01 and x != 0 else f"{x:.2f}",
)
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_rows", 100)

In [ ]:
SEGMENT_C = ["county", "product_type", "is_business"]
CATEGORICAL_C = ["county", "product_type", "is_business", "is_consumption"]
TARGET_C = [
    "county",
    "product_type",
    "is_business",
    "is_consumption",
    "datetime",
]
RAND = 10

In [ ]:
RAW_DATA_PATH = "../raw_data/"
ADDITIONAL_DATA_PATH = "../additional_data/"

# 2. Processing

In [ ]:
processed_dfs = process_all_dfs(
    load_all_raw_data(RAW_DATA_PATH, ADDITIONAL_DATA_PATH)
)

In [ ]:
processed_dfs.keys()

dict_keys(['train', 'gas_prices', 'client', 'electricity_prices', 'forecast_weather', 'historical_weather', 'station_county_mapping', 'county_id_to_name_map', 'holidays'])

In [ ]:
df = merge_all_dfs(processed_dfs, how="left")

In [ ]:
df = add_dst_flag(df)

In [ ]:
df = add_cyclic_datetime_features(df, drop_raw=True)

In [ ]:
# for lag in [2, 3, 7]:
for lag in range(2, 15):
    df = df.merge(
        get_lag(processed_dfs["train"][TARGET_C + ["target"]], lag=lag),
        on=TARGET_C,
    )

In [ ]:
# for window in [24, 24 * 3, 24 * 7, 24 * 14]:
for window in [24 * i for i in range(1, 15)]:
    df = df.merge(
        get_moving_average(
            processed_dfs["train"]
            .set_index("datetime")
            .sort_index()
            .groupby(CATEGORICAL_C, observed=True, as_index=False),
            window=window,
        ).dropna(),
        on=TARGET_C,
    )

In [ ]:
df["t_over_cap"] = (df["2d_lag_target"] / df["installed_capacity"]).astype(
    "float32"
)
df["t_over_eic"] = (df["2d_lag_target"] / df["eic_count"]).astype("float32")
df["cap_per_eic"] = (df["installed_capacity"] / df["eic_count"]).astype(
    "float32"
)

In [ ]:
# w, h = 20, 14

# fig, ax = plt.subplots(figsize=(w, h))
# # plt.figure(figsize=(w, h))

# sns.heatmap(
#     df.drop(
#         columns=df.select_dtypes(["category"]).columns.tolist()
#         + [
#             "datetime",
#             "data_block_id",
#             "date",
#         ]
#     ).corr(),
#     annot=True,
#     fmt=".1f",
#     annot_kws={"size": 7},
# )


# buf = io.BytesIO()
# fig.savefig(buf, format="png", bbox_inches="tight")
# buf.seek(0)
# img_base64 = base64.b64encode(buf.getvalue()).decode("utf-8")

# html_code = (
#     '<div style="overflow-x: auto; width: 100%;">'
#     '<img src="data:image/png;base64,{}" style="display: block; max-width: none; width: auto;">'
#     "</div>"
# ).format(img_base64)

# display(HTML(html_code))
# plt.close(fig)
# # plt.show()

# 3. Simple Baseline

In [ ]:
def split_by_equal_days(
    dt: Series,
    train_days: int = 1,
    fh_days: int = 1,  # Same for val and test
    val_steps: int = 1,
    expanding: bool = True,
) -> list[dict[str, tuple[pd.Timestamp, pd.Timestamp] | None]]:
    dt = dt.dt.floor("D")

    train_days = Timedelta(days=train_days)
    train_days_delta = train_days - Timedelta(days=1)  # Indexing from 0

    fh_days = Timedelta(days=fh_days)  # Indexing from 0
    fh_days_delta = fh_days - Timedelta(days=1)

    first_day = dt.min()
    last_day = dt.max()

    test_start = last_day - fh_days
    test_end = last_day + pd.Timedelta(hours=23)

    

    splits = []

    for step in range(val_steps + 1):  # val_steps times + 1 test-only
        if expanding:
            train_start = first_day
        else:
            train_start = first_day + step * train_days

        train_end = train_start + train_days_delta + pd.Timedelta(hours=23)

        val_start = train_end + pd.Timedelta(hours=1)
        val_end = val_start + fh_days_delta + pd.Timedelta(hours=23)

        if val_end >= test_start:
            # Skip creating val if it overlaps with test
            splits.append(
                {
                    "train": (train_start, train_end),
                    "val": None,
                    "test": (test_start, test_end),
                }
            )
            break

        splits.append(
            {
                "train": (train_start, train_end),
                "val": (val_start, val_end),
                "test": (test_start, test_end),
            }
        )

    return splits

In [ ]:
print(
    f"First timestamp: {df["datetime"].min()}",
    f"Last timestamp: {df["datetime"].max()}",
    sep="\n"
)

First timestamp: 2021-09-17 00:00:00
Last timestamp: 2023-05-31 23:00:00


Since the minimum usable lag for prediction is 48 hours, this horizon is particularly well suited as a naive baseline.

In [ ]:
splits_expanding = split_by_equal_days(
    df["datetime"],
    train_days=90,
    fh_days=1,  # Only daily predictions as per problem statement
    val_steps=4,  # Plus 1 without val
    expanding=True,
)

In [ ]:
val_mae = []
test_mae = []

for split in splits_expanding:
    print(split["train"])
    # val_slice = df.loc[
    #     (df["datetime"] >= split["val"][0])
    #     & (df["datetime"] <= split["val"][1]),
    #     ["target", "2d_lag_target"],
    # ]
    # test_slice = df.loc[
    #     (df["datetime"] >= split["test"][0])
    #     & (df["datetime"] <= split["test"][1]),
    #     ["target", "2d_lag_target"],
    # ]
    # val_mae.append(MAE(val_slice["target"], val_slice["2d_lag_target"]))
    # test_mae.append(MAE(test_slice["target"], test_slice["2d_lag_target"]))

print(val_mae)
print(test_mae)

(Timestamp('2021-09-17 00:00:00'), Timestamp('2021-12-15 23:00:00'))
(Timestamp('2021-09-17 00:00:00'), Timestamp('2021-12-15 23:00:00'))
(Timestamp('2021-09-17 00:00:00'), Timestamp('2021-12-15 23:00:00'))
(Timestamp('2021-09-17 00:00:00'), Timestamp('2021-12-15 23:00:00'))
(Timestamp('2021-09-17 00:00:00'), Timestamp('2021-12-15 23:00:00'))
[]
[]


In [ ]:
MAE(df["
"], df["target"])

81.69410705566406

In [ ]:
FEATURES_TO_DROP = ["datetime", "data_block_id", "date"]

In [ ]:
df["datetime"].is_monotonic_increasing
# df = df.sort_values('datetime', ignore_index=True)

True

In [ ]:
# for split in splits:
#     for k, v in split.items():
#         print(k, v, sep="\n")
#         print("\n")

In [ ]:
train_ts = "2022-06-01 00:00:00"
val_ts = "2022-07-01 00:00:00"
test_ts = "2022-08-01 00:00:00"

In [ ]:
# example_df = df.loc[df["datetime"] < test_timestamp]

In [ ]:
df_train = df[df["datetime"] < train_ts].drop(FEATURES_TO_DROP, axis=1)
df_val = df[
    (df["datetime"] >= train_ts) & (df["datetime"] < val_ts)
].drop(FEATURES_TO_DROP, axis=1)
df_test = df[
    (df["datetime"] >= val_ts) & (df["datetime"] < test_ts)
].drop(FEATURES_TO_DROP, axis=1)

X_train, y_train = df_train.drop(["target"], axis=1), df_train["target"]
X_val, y_val = df_val.drop(["target"], axis=1), df_val["target"]
X_test, y_test = df_test.drop(["target"], axis=1), df_test["target"]

In [ ]:
eval_set = [(X_train, y_train), (X_val, y_val)]

In [ ]:
n_estimators = 1000
max_depth=5
early_stopping_rounds = 100
learning_rate = 0.1

In [ ]:
xgbr = XGBRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    random_state=RAND,
    enable_categorical=True,
    eval_metric="mae",
    early_stopping_rounds=early_stopping_rounds,
    learning_rate=learning_rate,
)

In [ ]:
xgbr.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    verbose=True
)

In [ ]:
results = xgbr.evals_result()

plt.plot(results["validation_0"]["mae"], label="Training loss")
plt.plot(results["validation_1"]["mae"], label="Validation loss")

plt.xlabel("Number of estimators")
plt.ylabel("Loss")

plt.legend()
# plt.yscale("log")
plt.grid(which='both')

plt.show()

In [ ]:
y_pred = xgbr.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
residuals = y_test - y_pred

plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.title("Residuals vs Predicted")
plt.show()

In [ ]:
fi = pd.Series(xgbr.feature_importances_, index=X_train.columns)
fi = fi.sort_values(ascending=False).head(20)

fi.plot.barh(figsize=(8,6))
plt.xlabel("Feature Importance")
plt.gca().invert_yaxis()
plt.title("Top 20 Important Features")
plt.show()